In [ ]:
# Loading libraries
import numpy as np
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format='retina'
from matplotlib import pyplot as plt
import pandas as pd
import h5py

import bilby
from hanabi.hierarchical.source_population_model import *
from hanabi.hierarchical.merger_rate_density import *
from hanabi.hierarchical.selection_function import *
from hanabi.hierarchical.p_z import *
from hanabi.lensing.optical_depth import *
from hanabi.lensing.absolute_magnification import *
from hanabi.hierarchical.marginalized_likelihood import *
from hanabi.hierarchical.gibbs_sampling import CustomCollapsedBlockedGibbsSampler
from hanabi.hierarchical.reweight_with_population_model import *

In [ ]:
defaults_kwargs = {
    'bins': 50,
    'smooth': 0.9,
    'label_kwargs': {'fontsize': 16},
    'title_kwargs': {'fontsize': 16},
    'color': '#0072C1',
    'truth_color': 'tab:orange',
    'quantiles': [0.16, 0.84],
    'levels': (1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.)),
    'plot_density': False,
    'plot_datapoints': True,
    'fill_contours': True,
    'max_n_ticks': 3
}

In [ ]:
# Loading PE results
rundir = "RUNDIR"
event_1 = "EVENT_1"
event_2 = "EVENT_2"

single_event_PE_result_path_template = "{rundir}/{event}_0_reweighted_result.json"
joint_PE_result_path_template = "{rundir}/joint_{event_1}_{event_2}_0_reweighted_result.json"

result_1 = bilby.result.read_in_result(single_event_PE_result_path_template.format(rundir=rundir, event=event_1))
result_2 = bilby.result.read_in_result(single_event_PE_result_path_template.format(rundir=rundir, event=event_2))
joint_result = bilby.result.read_in_result(joint_PE_result_path_template.format(rundir=rundir, event_1=event_1, event_2=event_2))

In [ ]:
def log_coherence_ratio(joint_result, *single_result):
    joint_log_evidence = joint_result.log_evidence
    return joint_log_evidence - np.sum([r.log_evidence for r in list(single_result)])

lcr = log_coherence_ratio(joint_result, result_1, result_2)
print(lcr)

In [ ]:
# Setting up the population models
mass_src_pop_model = PowerLawComponentMass(alpha=1.8, beta=0, mmin=5, mmax=60)
spin_src_pop_model = UniformAlignedSpinComponent()
merger_rate_density = BelczynskiEtAl2017PopIPopIIStarsBBHMergerRateDensity()

# Setting up the strong lensing models
optical_depth = HannukselaEtAl2019OpticalDepth()
pz_lensed = LensedSourceRedshiftProbDist(merger_rate_density=merger_rate_density, optical_depth=optical_depth)
pz_notlensed = NotLensedSourceRedshiftProbDist(merger_rate_density=merger_rate_density, optical_depth=optical_depth)
abs_magnifications = [SISPowerLawAbsoluteMagnification(), SISPowerLawAbsoluteMagnification()]

In [ ]:
ncores = 8
label = "marginalize_over_redshift_{}_{}".format(event_1, event_2)

In [ ]:
# Setting up likelihood
likelihood = MonteCarloMarginalizedLikelihood(joint_result, mass_src_pop_model, spin_src_pop_model, abs_magnifications)
# Marginalizing over redshift for the joint result
redshift_result = bilby.run_sampler(likelihood=likelihood, priors={'redshift': pz_lensed}, sampler='dynesty', nlive=1000, nact=30, npool=ncores, sample='unif', outdir="marginalization", label=label)

In [ ]:
# Reweighting single-event results with population models
reweighting_result_1 = ReweightWithPopulationModel(result_1, mass_src_pop_model, spin_src_pop_model, pz_notlensed)
reweighting_result_2 = ReweightWithPopulationModel(result_2, mass_src_pop_model, spin_src_pop_model, pz_notlensed)

In [ ]:
# Calculating the proper Bayes factor
with h5py.File("/home/ka-lok.lo/projects/stronglensing/hanabi/hanabi/hierarchical/pdetclassifier/aLIGOdesign_bbhpop_powerlaw.h5", "r") as f:
    log_alpha_sq_over_beta = np.log(f.attrs["alpha"]**2/f.attrs["beta"])

log_evidence_ratio = redshift_result.log_evidence - (reweighting_result_1.reweight_ln_evidence() + reweighting_result_2.reweight_ln_evidence())
print(log_alpha_sq_over_beta + log_evidence_ratio)

In [ ]:
print(log_alpha_sq_over_beta)

In [ ]:
print(log_evidence_ratio)

In [ ]:
# Gibbs sampling
sampler = CustomCollapsedBlockedGibbsSampler(redshift_result, likelihood, pool=ncores)
joint_samples = sampler.sample(10000)

In [ ]:
joint_samples["magnification^(1)"] = (bilby.gw.conversion.redshift_to_luminosity_distance(joint_samples["redshift"])/joint_samples["luminosity_distance^(1)"])**2
joint_samples["magnification^(2)"] = (bilby.gw.conversion.redshift_to_luminosity_distance(joint_samples["redshift"])/joint_samples["luminosity_distance^(2)"])**2
joint_samples["relative_magnification"] = joint_samples["magnification^(2)"]/joint_samples["magnification^(1)"]
joint_samples["mass_1_source"] = joint_samples["mass_1"]/(1. + joint_samples["redshift"])
joint_samples["mass_2_source"] = joint_samples["mass_2"]/(1. + joint_samples["redshift"])
joint_samples["total_mass_source"] = joint_samples["mass_1_source"] + joint_samples["mass_2_source"]

In [ ]:
import corner
plt.rcParams.update({'text.usetex': True})
plt.figure(dpi=150)
fig = corner.corner(joint_samples[["mass_1_source", "mass_2_source", "total_mass_source", "magnification^(1)", "magnification^(2)", "relative_magnification", "redshift"]], labels=[r'$m_1^{\rm src}\;[M_{\odot}]$', r'$m_2^{\rm src}\;[M_{\odot}]$', r'$M_{\rm tol}^{\rm src}\;[M_{\odot}]$', r'$\mu^{(1)}$', r'$\mu^{(2)}$', r'$\mu^{(\rm rel)}$', r'$z$'], **defaults_kwargs)